<big style="font-size:2em;color:yellow;background-color:black;">
**🌎|🌎**   <big style="font-size:0.7em;color:brown;background-color:white;padding:-2pt"> 
&nbsp; Streaming With OpenCV &nbsp;
</big>
</big>

With a simple webcam, we can create a cross-platform of face-recognitioon web app.



**1.** File structure of App:

<pre>
   🔻[<font color="blue">📂 App Folder</font>]
         🔻 [<font color="blue">📂 templates</font>] 
                🔻[<font color="blue">📂 imgs</font>]
                🗒 index.html
         🔻 [<font color="blue">📂 imgs</font>]
                 me.jpg 
         🗒 app.py    <font color="brown">(basic streaming app)</font>
         🗒 app-2.py  <font color="brown">(implement facial recognition function )</font>
</pre>


<b>2. app.py</b> is the main code of the web app:
```python
#!/usr/bin/env python
from flask import Flask, render_template, Response
import cv2

app = Flask(__name__)
vc = cv2.VideoCapture(0)

@app.route('/')
def index():
    """Video streaming home page."""
    return render_template('index.html')

def gen():
    """Video streaming generator function."""
    imgfile='templates/imgs/t.jpg';
    while True:
        rval, frame = vc.read()
        cv2.imwrite(imgfile, frame)
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + open(imgfile, 'rb').read() + b'\r\n')

@app.route('/video_feed')
def video_feed():
    """Video streaming route. Put this in the src attribute of an img tag."""
    return Response(gen(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True, threaded=True)
```
Note
---
1. app.py acclaims two sites prepared for services:<br>
   a). "http://127.0.0.1:5000/" for the main home page;<br>
   b). "http://127.0.0.1:5000/vieo_feed" for page of streaming data return. 
- "yield", as iterator, which could optimize resource of returning data.
- ```flask.Responce()```, returns an object, having HTML mimetype.

**3.** index.html: Home Page,
```html
<html>
  <head>
     <title>Stream Face-recognition App</title>
  </head>
  <body>
    <img src="{{url_for('video_feed')}}" />
  </body>
</html>
```

Now, startup the app by:
```
> python app.py
```
and visit local host, <font color="blue">127.0.0.1:5000</font>, for testing! 

While running app, the complete code of page is as follows:

```html
<html>
  <head>
    <title>Video Streaming Demonstration</title>
  </head>
  <body>
    <h1>Video Streaming Demonstration</h1>
    <img src="/video_feed" />
  </body>
</html>

```
This means, app reads each frame, at which does face-dectect, makes identification, saves image, refresh frame, and repeat the procedure. 

## Face-Recognition on Video, again

With the experience of doing Face-Recognition last time, we can binding these two into one, <font color="brown">app-2.py</font>, which could identify person in video online:

```python
#!/usr/bin/env python
from flask import Flask, render_template, Response
import face_recognition
import cv2

app = Flask(__name__)
video_capture = cv2.VideoCapture(0)

# Load a sample picture and learn how to recognize it.
me_image = face_recognition.load_image_file("imgs/me.jpg")
me_face_encoding = face_recognition.face_encodings(me_image)[0]

@app.route('/')
def index():
    """Video streaming home page."""
    return render_template('index.html')


def gen():
    """Video streaming generator function."""
    imgfile='templates/imgs/test.jpg';
    
    # Initialize some variables
    face_locations = []
    face_encodings = []
    face_names = []
    process_this_frame = True
    
    while True:
        rval, frame = video_capture.read()
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        
        # Only process every other frame of video to save time
        if process_this_frame:
           # Find all the faces and face encodings in the current frame of video
           face_locations = face_recognition.face_locations(small_frame)
           face_encodings = face_recognition.face_encodings(small_frame, face_locations)

           face_names = []
           for face_encoding in face_encodings:
               # See if the face is a match for the known face(s)
               match = face_recognition.compare_faces([me_face_encoding], face_encoding)
               name = "Unknown"

               if match[0]:
                  name = "ME"

               face_names.append(name)

        process_this_frame = not process_this_frame
        
        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        
            cv2.imwrite(imgfile, frame)
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + open(imgfile, 'rb').read() + b'\r\n')

@app.route('/video_feed')
def video_feed():
    """Video streaming route. Put this in the src attribute of an img tag."""
    return Response(gen(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True, threaded=True)

```

Final Note
---
We can make an app of face-recognition for mobile devices; for instance, by "webview" object of Android.

In [ ]:
from PIL import Image,ImageDraw,ImageFont